# Imports para Spark

In [ ]:
from pyspark.sql import HiveContext
from pyspark.sql.types import *

# Operações HDFS 

In [ ]:
!hdfs dfs -ls /datasus

In [ ]:
!hdfs dfs -ls /user/hive/warehouse

In [ ]:
!hdfs dfs -copyFromLocal /input/*.csv /datasus/

!hdfs dfs -rm /datasus/*.csv

In [ ]:
!hdfs dfs -rm -R /user/hive/warehouse/covid19

!hdfs dfs -rm -R /user/hive/warehouse/covid19_particionada_codmun/*

# Spark context

In [18]:
spark.sql("show tables").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|             covid19|      false|
| default|         covid19_opt|      false|
| default|   covid19_otimizada|      false|
| default|customer_transact...|      false|
|        |   covid19_temporary|       true|
+--------+--------------------+-----------+



In [ ]:
sc = spark.sparkContext

In [ ]:
sqlContext = HiveContext(sc)

df = spark.read.csv("/datasus/HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv", header="true", sep=";", quote="\'", inferSchema=True)

df = spark.read.csv("/datasus/HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv", header="true", sep=";", quote="\'", inferSchema=True)

df = spark.read.csv("/datasus/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv", header="true", sep=";", quote="\'", inferSchema=True)

df = spark.read.csv("/datasus/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv", header="true", sep=";", quote="\'", inferSchema=True)

In [11]:
df = spark.read.csv("/datasus/", header="true", sep=";", quote="\'", inferSchema=True)

In [12]:
df = df.na.fill(value=0)

In [13]:
df.createOrReplaceTempView("covid19_temporary") 

In [16]:
spark.sql("create table covid19_opt as select * from covid19_temporary");

df = spark.read.table('covid19')

In [ ]:
df.count()

In [ ]:
df.show(n=20, truncate=False, vertical=True )

In [ ]:
df.printSchema()

df.na.fill(0, ["codmun"]).show()

df.na.fill(value=0,subset=["codmun"]).show()

In [ ]:
df = df.na.fill(value=0)

# register as a temporary view [sql]
df.createOrReplaceTempView("df_view")

df.write.partitionBy("codmun") \
        .mode("append") \
        .save("hdfs://namenode:8020/datasus/covid19_particionada_codmun2")

df.write.save("hdfs://namenode:8020/datasus/covid19_particionada_codmun3")

df.write.partitionBy("codmun").saveAsTable("covid19_particionada_codmun")

spark.sql("SELECT codmun,count FROM df_view")

In [ ]:
topic_string = df.select("regiao","coduf","codmun")
topic_string.show(truncate=False)

In [ ]:
sqlContext.setConf("hive.exec.dynamic.partition", "true")
sqlContext.setConf("hive.exec.dynamic.partition.mode", "nonstrict")

In [ ]:
df.write.partitionBy("codmun") \
        .mode("append") \
        .saveAsTable("covid19_particionada_codmun")

In [ ]:
spark.sql("SELECT count(*) FROM covid19.parquet c19").show()

# primeira visualização

In [ ]:
spark.sql("select max(data) from covid19_opt")

In [ ]:
col_list = df.select('mycol').collect()

In [ ]:
spark.sql("SELECT data,count(1),sum(c19.Recuperadosnovos),sum(c19.emAcompanhamentoNovos) FROM covid19 c19 where c19.regiao like 'Brasil%' group by data order by data").show(400)

In [ ]:
spark.sql("SELECT c19.regiao,count(1) FROM covid19 c19 group by c19.regiao").show()

In [ ]:
spark.sql("SELECT c19.municipio,c19.estado,c19.codmun,c19.Recuperadosnovos,c19.emAcompanhamentoNovos FROM covid19 c19 where c19.codmun = 150300").show(10000)

In [ ]:
spark.sql("SELECT c19.municipio,count(1) FROM covid19 c19 group by c19.municipio").show(10000)

# segunda visualização